In [2]:
import numpy as np
import pandas as pd

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
pd.set_option('display.max_columns', 200)

In [10]:
movies = pd.read_csv('movies.csv')
print("movies.csv")
display(movies.head(1))

reviews = pd.read_csv('reviews.csv')
print("reviews.csv")
display(reviews.head(1))

search_logs = pd.read_csv('search_logs.csv')
print("search_logs.csv")
display(search_logs.head(1))

users = pd.read_csv('users.csv')
print("users.csv")
display(users.head(1))

watch_history = pd.read_csv('watch_history.csv')
print("watch_history.csv")
display(watch_history.head(1))

movies.csv


,movie_id,title,content_type,genre_primary,genre_secondary,release_year,duration_minutes,rating,language,country_of_origin,imdb_rating,production_budget,box_office_revenue,number_of_seasons,number_of_episodes,is_netflix_original,added_to_platform,content_warning
0,movie_0001,Dragon Legend,Stand-up Comedy,History,Thriller,2014,35.0,TV-Y,French,Japan,NaN,NaN,NaN,NaN,NaN,False,2023-08-07,False


reviews.csv


,review_id,user_id,movie_id,rating,review_date,device_type,is_verified_watch,helpful_votes,total_votes,review_text,sentiment,sentiment_score
0,review_000001,user_07066,movie_0360,4,2025-03-29,Mobile,False,3.0,5.0,Fantastic cinematography and plot twists.,positive,0.711


search_logs.csv


,search_id,user_id,search_query,search_date,results_returned,clicked_result_position,device_type,search_duration_seconds,had_typo,used_filters,location_country
0,search_000001,user_09864,classic movies,2024-03-22,20,2.0,Tablet,12.4,False,False,Canada


users.csv


,user_id,email,first_name,last_name,age,gender,country,state_province,city,subscription_plan,subscription_start_date,is_active,monthly_spend,primary_device,household_size,created_at
0,user_00001,figueroajohn@example.org,Erica,Garza,43.0,Male,USA,Massachusetts,North Jefferyhaven,Basic,2024-04-08,True,36.06,Laptop,1.0,2023-04-01 14:40:50.540242


watch_history.csv


,session_id,user_id,movie_id,watch_date,device_type,watch_duration_minutes,progress_percentage,action,quality,location_country,is_download,user_rating
0,session_000001,user_07271,movie_0511,2025-11-13,Tablet,63.9,34.6,completed,HD,USA,False,NaN


**User Based Based Collaborative Filtering**

---


Finding similar users and recommending what similar users have liked and not watched by a particular user

In [93]:
#create user - movie matrix, where rows have user, columns have movie id and values as ratings
user_matrix = pd.pivot_table(reviews, index = 'user_id', columns = 'movie_id', values = 'rating')
display(user_matrix.head())
print(user_matrix.shape)

movie_id,movie_0001,movie_0002,movie_0003,movie_0004,movie_0005,movie_0006,movie_0007,movie_0008,movie_0009,movie_0010,movie_0011,movie_0012,movie_0013,movie_0014,movie_0015,movie_0016,movie_0017,movie_0018,movie_0019,movie_0020,movie_0021,movie_0022,movie_0023,movie_0024,movie_0025,movie_0026,movie_0027,movie_0028,movie_0029,movie_0030,movie_0031,movie_0032,movie_0033,movie_0034,movie_0035,movie_0036,movie_0037,movie_0038,movie_0039,movie_0040,movie_0041,movie_0042,movie_0043,movie_0044,movie_0045,movie_0046,movie_0047,movie_0048,movie_0049,movie_0050,movie_0051,movie_0052,movie_0053,movie_0054,movie_0055,movie_0056,movie_0057,movie_0058,movie_0059,movie_0060,movie_0061,movie_0062,movie_0063,movie_0064,movie_0065,movie_0066,movie_0067,movie_0068,movie_0069,movie_0070,movie_0071,movie_0072,movie_0073,movie_0074,movie_0075,movie_0076,movie_0077,movie_0078,movie_0079,movie_0080,movie_0081,movie_0082,movie_0083,movie_0084,movie_0085,movie_0086,movie_0087,movie_0088,movie_0089,movie_0090,movie_0091,movie_0092,movie_0093,movie_0094,movie_0095,movie_0096,movie_0097,movie_0098,movie_0099,movie_0100,...,movie_0901,movie_0902,movie_0903,movie_0904,movie_0905,movie_0906,movie_0907,movie_0908,movie_0909,movie_0910,movie_0911,movie_0912,movie_0913,movie_0914,movie_0915,movie_0916,movie_0917,movie_0918,movie_0919,movie_0920,movie_0921,movie_0922,movie_0923,movie_0924,movie_0925,movie_0926,movie_0927,movie_0928,movie_0929,movie_0930,movie_0931,movie_0932,movie_0933,movie_0934,movie_0935,movie_0936,movie_0937,movie_0938,movie_0939,movie_0940,movie_0941,movie_0942,movie_0943,movie_0944,movie_0945,movie_0946,movie_0947,movie_0948,movie_0949,movie_0950,movie_0951,movie_0952,movie_0953,movie_0954,movie_0955,movie_0956,movie_0957,movie_0958,movie_0959,movie_0960,movie_0961,movie_0962,movie_0963,movie_0964,movie_0965,movie_0966,movie_0967,movie_0968,movie_0969,movie_0970,movie_0971,movie_0972,movie_0973,movie_0974,movie_0975,movie_0976,movie_0977,movie_0978,movie_0979,movie_0980,movie_0981,movie_0982,movie_0983,movie_0984,movie_0985,movie_0986,movie_0987,movie_0988,movie_0989,movie_0990,movie_0991,movie_0992,movie_0993,movie_0994,movie_0995,movie_0996,movie_0997,movie_0998,movie_0999,movie_1000
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
user_00001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
user_00002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

(7761, 1000)


In [94]:
#get user - user similarity
user_similarity = cosine_similarity(user_matrix.fillna(0))
user_similarity

#make pandas df of this
user_similarity_df = pd.DataFrame(user_similarity, index = user_matrix.index, columns = user_matrix.index)
user_similarity_df.head()

user_id,user_00001,user_00002,user_00003,user_00004,user_00005,user_00006,user_00007,user_00010,user_00011,user_00013,user_00014,user_00015,user_00016,user_00017,user_00018,user_00019,user_00020,user_00021,user_00023,user_00024,user_00025,user_00026,user_00027,user_00028,user_00029,user_00030,user_00032,user_00033,user_00034,user_00035,user_00036,user_00038,user_00039,user_00040,user_00041,user_00043,user_00044,user_00045,user_00046,user_00047,user_00048,user_00049,user_00050,user_00051,user_00052,user_00055,user_00056,user_00057,user_00058,user_00059,user_00060,user_00063,user_00065,user_00067,user_00068,user_00069,user_00071,user_00072,user_00073,user_00075,user_00076,user_00077,user_00078,user_00079,user_00080,user_00081,user_00082,user_00083,user_00084,user_00088,user_00089,user_00091,user_00093,user_00094,user_00095,user_00097,user_00098,user_00099,user_00100,user_00101,user_00102,user_00103,user_00105,user_00106,user_00107,user_00110,user_00111,user_00112,user_00113,user_00114,user_00115,user_00117,user_00118,user_00119,user_00120,user_00121,user_00123,user_00124,user_00125,user_00128,...,user_09875,user_09877,user_09878,user_09879,user_09880,user_09881,user_09882,user_09883,user_09884,user_09885,user_09886,user_09888,user_09889,user_09890,user_09891,user_09892,user_09893,user_09894,user_09896,user_09898,user_09900,user_09901,user_09902,user_09903,user_09904,user_09905,user_09906,user_09908,user_09909,user_09910,user_09911,user_09912,user_09913,user_09915,user_09916,user_09917,user_09918,user_09920,user_09922,user_09923,user_09927,user_09928,user_09930,user_09931,user_09932,user_09933,user_09935,user_09936,user_09937,user_09938,user_09939,user_09940,user_09941,user_09943,user_09944,user_09945,user_09946,user_09947,user_09949,user_09950,user_09952,user_09953,user_09955,user_09956,user_09957,user_09958,user_09961,user_09962,user_09964,user_09965,user_09966,user_09967,user_09970,user_09971,user_09972,user_09973,user_09974,user_09975,user_09977,user_09978,user_09979,user_09980,user_09981,user_09982,user_09983,user_09984,user_09985,user_09986,user_09987,user_09988,user_09989,user_09990,user_09991,user_09993,user_09994,user_09995,user_09997,user_09998,user_09999,user_10000
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
user_00001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
user_00002,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

Recommending

In [98]:

user_id = 'user_09033'
similar_user = user_similarity_df[user_id].sort_values(ascending = False)
similar_user = similar_user.drop(user_id)
top_similar_user = similar_user.head().index

# get top 5 similar users (their user_ids)
top_similar_user = similar_user.head().index

#filtering user_matrix for these users
neighbor_rating = user_matrix.loc[top_similar_user]

weighted_ratings = (neighbor_rating.T * similar_user[top_similar_user]).T  # broadcast similarity
weighted_ratings = weighted_ratings.sum(axis=0) / similar_user[top_similar_user].sum()

# Remove movies already watched by the target user
already_watched = user_matrix.loc[user_id].dropna().index
recommendation = weighted_ratings.drop(already_watched, errors='ignore')

# Get Top-N recommendations
top10 = recommendation.sort_values(ascending=False).head(10)
print("Top 10 recommended movies for", user_id)
print(top10)

Top 10 recommended movies for user_09033
movie_id
movie_1000    0.0
movie_0001    0.0
movie_0002    0.0
movie_0003    0.0
movie_0004    0.0
movie_0005    0.0
movie_0006    0.0
movie_0984    0.0
movie_0983    0.0
movie_0982    0.0
dtype: float64


### Item Based Collaborative Filtering

In [78]:
item_matrix = pd.pivot_table(reviews, index = 'user_id', columns = 'movie_id', values = 'rating')
item_matrix.shape

(7761, 1000)

In [80]:
item_similarity = cosine_similarity(item_matrix.T.fillna(0))
item_similarity_df = pd.DataFrame(item_similarity, index = item_matrix.columns, columns = item_matrix.columns)
item_similarity_df.head()

movie_id,movie_0001,movie_0002,movie_0003,movie_0004,movie_0005,movie_0006,movie_0007,movie_0008,movie_0009,movie_0010,movie_0011,movie_0012,movie_0013,movie_0014,movie_0015,movie_0016,movie_0017,movie_0018,movie_0019,movie_0020,movie_0021,movie_0022,movie_0023,movie_0024,movie_0025,movie_0026,movie_0027,movie_0028,movie_0029,movie_0030,movie_0031,movie_0032,movie_0033,movie_0034,movie_0035,movie_0036,movie_0037,movie_0038,movie_0039,movie_0040,movie_0041,movie_0042,movie_0043,movie_0044,movie_0045,movie_0046,movie_0047,movie_0048,movie_0049,movie_0050,movie_0051,movie_0052,movie_0053,movie_0054,movie_0055,movie_0056,movie_0057,movie_0058,movie_0059,movie_0060,movie_0061,movie_0062,movie_0063,movie_0064,movie_0065,movie_0066,movie_0067,movie_0068,movie_0069,movie_0070,movie_0071,movie_0072,movie_0073,movie_0074,movie_0075,movie_0076,movie_0077,movie_0078,movie_0079,movie_0080,movie_0081,movie_0082,movie_0083,movie_0084,movie_0085,movie_0086,movie_0087,movie_0088,movie_0089,movie_0090,movie_0091,movie_0092,movie_0093,movie_0094,movie_0095,movie_0096,movie_0097,movie_0098,movie_0099,movie_0100,...,movie_0901,movie_0902,movie_0903,movie_0904,movie_0905,movie_0906,movie_0907,movie_0908,movie_0909,movie_0910,movie_0911,movie_0912,movie_0913,movie_0914,movie_0915,movie_0916,movie_0917,movie_0918,movie_0919,movie_0920,movie_0921,movie_0922,movie_0923,movie_0924,movie_0925,movie_0926,movie_0927,movie_0928,movie_0929,movie_0930,movie_0931,movie_0932,movie_0933,movie_0934,movie_0935,movie_0936,movie_0937,movie_0938,movie_0939,movie_0940,movie_0941,movie_0942,movie_0943,movie_0944,movie_0945,movie_0946,movie_0947,movie_0948,movie_0949,movie_0950,movie_0951,movie_0952,movie_0953,movie_0954,movie_0955,movie_0956,movie_0957,movie_0958,movie_0959,movie_0960,movie_0961,movie_0962,movie_0963,movie_0964,movie_0965,movie_0966,movie_0967,movie_0968,movie_0969,movie_0970,movie_0971,movie_0972,movie_0973,movie_0974,movie_0975,movie_0976,movie_0977,movie_0978,movie_0979,movie_0980,movie_0981,movie_0982,movie_0983,movie_0984,movie_0985,movie_0986,movie_0987,movie_0988,movie_0989,movie_0990,movie_0991,movie_0992,movie_0993,movie_0994,movie_0995,movie_0996,movie_0997,movie_0998,movie_0999,movie_1000
movie_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
movie_0001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.029486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
movie_0002,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.061981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.044193,0.000000,0.0,0.0,0.0,0.0,0.051066,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000

In [91]:
target_user= 'user_01177'

# get ratings for target user
user_ratings = item_matrix.loc[target_user].dropna()

# similarities for movies rated by user
similarities = item_similarity_df.loc[user_ratings.index]

# weighted score (matrix multiply)
weighted_score = similarities.T @ user_ratings
weighted_score = weighted_score / user_ratings.sum()

# remove already watched movies
recommendation = weighted_score.drop(user_ratings.index)

# Top 10 recommendations
top10 = recommendation.sort_values(ascending=False).head(10)
print(top10)

movie_id
movie_0313    0.052917
movie_0425    0.046184
movie_0766    0.028265
movie_0949    0.027988
movie_0807    0.023610
movie_0303    0.022990
movie_0128    0.022848
movie_0511    0.022389
movie_0708    0.021402
movie_0330    0.020753
dtype: float64
